Ce challenge aborde le cryptosystème **LWE** *(Learning With Errors)* (Voir https://fr.wikipedia.org/wiki/Apprentissage_avec_erreurs). Cependant, on se rend après quelques lectures du programme que cette version de **LWE** est une version alternative s'appellant **ILWE** *(Integer Learning With Errors)* qui est une version de **LWE** sans réduction modulaire. On a donc le problème ILWE devant nous qui s'écrit plus formellement: 

Disposant de $m$ échantillons de la forme $(a_i,_bi)$,


In [37]:
from pwn import *
from Crypto.Random.random import getrandbits
import re

host = '0.0.0.0'
port = '4000'

r = remote(host,port)
n = 64

def get_n_samples(r,n):
    
    assert n > 0
    
    command = 's'
    A = []
    B = []
    
    r.recv()
    for i in range(n):
        r.sendline(command.encode())
        data = r.recvuntil(b'>>>').decode()
        sample_a = vector(ZZ,list(map(int, re.search(r'\[(.*?)\]', data).group(1).split(', '))))
        sample_b = Integer(re.search(r'\]\n(\d+)', data).group(1))
        A.append(sample_a)
        B.append(sample_b)
            
    return Matrix(ZZ,A), vector(ZZ,B)
    
A,B = get_n_samples(r,n)
M=(identity_matrix(n).augment(A.transpose())).stack(vector([0]*n + list(-B)))  
reduced_lattice = M.LLL()
secret = reduced_lattice.row(-1)[:n]

r.sendline(b'f')
r.recvuntil(b'Enter the server secret: ')
r.sendline(str(list(secret)).encode())
print(r.recv().decode())
r.close()

[x] Opening connection to 0.0.0.0 on port 4000
[x] Opening connection to 0.0.0.0 on port 4000: Trying 0.0.0.0
[+] Opening connection to 0.0.0.0 on port 4000: Done
Congrats!! Here is the flag: FCSC{b50ceb3b41dbc0bcb50a54f9bac51bcff34856f5a17346dd6754673355609c48}

[*] Closed connection to 0.0.0.0 port 4000


In [85]:
e =  130
n =  64
q = 128

def randomize_secret(n):
	return [ getrandbits(q) for _ in range(n) ]

def dot(a, b):
	return sum(x * y for x, y in zip(a, b))

def LWE(s):
	a = [ getrandbits(q) for _ in range(n)]
	return (a, dot(a, s) + getrandbits(e))



import math

def calculate_standard_deviation(q, e):
    sigma_a = (2 ** q) / math.sqrt(12)
    sigma_e = (2 ** e) / math.sqrt(12)
    return sigma_a, sigma_e

def calculate_samples_needed(n, sigma_a, sigma_e):
    m = math.log2(n) * (sigma_e / sigma_a) ** 2
    return m


sigma_a, sigma_e = calculate_standard_deviation(q, e)
m = calculate_samples_needed(n, sigma_a, sigma_e)


print(m)

96.0


In [86]:
s = randomize_secret(n)
A = []
B = []
for i in range(3*n):
    a,b = LWE(s)
    A.append(a)
    B.append(b)

A = matrix(ZZ,A)
B = vector(ZZ,B)

produit = matrix(RR,A.transpose()*A)

assert produit.is_invertible()

S = produit.inverse()*A.transpose()*B


In [87]:
print(S[0]-s[0])

-3.03364822859546e25
